# TORCH.AUTOGRAD를 사용한 자동 미분기

신경망을 학습 시 자주 사용되는 알고리즘은 Back-propagation이다.  
Back-propagation에서 매개변수(모델의 weight)는 주어진 매개변수에 대한 loss function의 gradient(변화도)에 따라 조정된다.  
  
  이러한 gradient를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있다. torch.autograd는 모든 계산 그래프에 대한 gradient를 자동 계산을 지원한다.  
    
    입력 x, 매개변수 w와 b, 그리고 일부 loss function이 있는 가장 간단한 단일 계층 신경망을 가정하겠다.

In [3]:
from importlib.metadata import requires
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad= True)
b = torch.randn(3, requires_grad= True)
z = torch.matmul(x, w) + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

# Tensor, Function과 Computational graph (연산그래프)

![](/Users/junghwankim/Desktop/pytorch_tutorial/img1.jpg)  

  
  이 신경망에서 w와 b는 최적화를 해야 하는 매개변수이다. 따라서, 이러한 변수들에 대한 loss function의 gradient를 계산할 수 있어야 한다. 이를 위해서 해당 tensor에 requires_grad 속성을 설정한다.

requires_grad의 값은 tensor를 생성할 때 설정하거나, 나중에 x.requires_grad_(True) 메소드를 사용하여 설정할 수도 있다.

연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 Function class의 object이다. 이 object는 forward-propagation으로 function을 계산하는 방법과 back-propagation 단계에서 derivative(도함수)를 계산하는 방법을 알고 있다. back-propagation function에 대한 reference는 tensor의 grad_fn 속성에 저장된다.

In [9]:
print(f'Gradient function for z = {z.grad_fn}')
print(f'Gradient function for loss = {loss.grad_fn}')


Gradient function for z = <AddBackward0 object at 0x12750a970>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x11ec01640>


# Gradient(변화도) 계산하기

신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 loss function의 derivative를 계산해야 한다. 즉, x와 y의 일부 고정 값에서 a(loss)/a(w)와 a(loss)/a(b)가 필요하다.  
  
  이런 도함수를 계산하기 위해, loss.backward()를 호출하고 w.grad와 b.grad에서 값을 가져와야 한다.

In [10]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3317, 0.1407, 0.3139],
        [0.3317, 0.1407, 0.3139],
        [0.3317, 0.1407, 0.3139],
        [0.3317, 0.1407, 0.3139],
        [0.3317, 0.1407, 0.3139]])
tensor([0.3317, 0.1407, 0.3139])


연산 그래프의 leaf node들 중 requires_grad 속성이 True로 설정된 노드들의 grad만 구할 수 있다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않다.  
  
    
성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행 가능하다. 만약 동일한 그래프에서 여러번 backward가 필요하면, backward 호출 시에 retrain_graph = True를 전달해야 한다.

# 변화도 추적 멈추기

기본적으로, requires_grad=True인 모든 tensor들은 연산 기록을 추적하고 변화도 계산을 지원한다. 그러나, 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 forward-propagation만 필요한 경우, 이러한 추적이나 지원이 필요 없을 수 있다.  
  

연산 코드를 torch.no_grad() 블록으로 감싸 연산 추적을 멈출 수 있다.

In [11]:
z = torch.matmul(x,w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b

print(z.requires_grad)

True
False


동일한 결과를 얻는 다른 방법은 텐서에 detach() 메소드를 사용하는 것이다.

In [12]:
z = torch.matmul(x,w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


# 변화도 추적을 멈춰야 하는 이유

1. 신경망의 일부 매개변수를 고정된 매개변수로 표시 한다. 이는 pre-trained된 신경망을 fine-tunning 시 매우 일반적인 시나리오다.
2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, forward-propagation만 수행할 때 연산 속도가 향상된다.

# 연산 그래프에 대한 추가 정보

개념적으로, autograd는 데이터(텐서)의 실행된 모든 연산들(연산 결과가 새로운 텐서인 경우도 포함하여)의 기록을 Function 객체로 구성된 방향성 비순환 그래프(DAG : Directed Acyclic Graph)에 저장합니다. 이 DAG의 leaf들은 입력 텐서이고, root(뿌리)는 결과 텐서이다. 이 그래프를 뿌리에서 잎까지 추적하면 chain rule에 따라 변화도를 자동으로 계산할 수 있다.  

순전파 단계에서, autograd는 다음 두 가지 작업을 동시에 수행한다.  
  
- 요청된 연산을 수행하여 결과 텐서를 계산한다.
- DAG에 연산의 변화도 기능(gradient function)을 유지 한다.

역전파 단계는 DAG에서 .backward()가 호출될 때 시작된다.
autograd는 이떄:  
  
- 각 .grad_fn으로부터 변화도를 계산
- 각 텐서의 .grad 속성에 계산 결과를 accumulate
- chain rule을 사용하여, 모든 잎 텐서들까지 propagate한다.

# Pytorch에서 DAG들은 dynamic이다.

그래프가 처음부터 다시 생성된다는 점에 주목해야 함.
매번 .backward()가 호출되고 나면 autograd는 새로운 그래프를 채우기 시작한다.  
이러한 점 덕분에 모델에서 흐름 제어 구문들을 사용할 수 있는 것이다.  
매번 iteration할 때 마다 필요하면 모양이나 크기 연산을 바꿀 수 있다.

# Optional Reading : 텐서 변화도와 야코비안 곱 (ja cobian product)

대부분의 경우 스칼라 손실 함수를 가지고 일부 매개변수와 관련된 변화도를 계산해야 한다. 그러나 출력 함수가 임의의 텐서인 경우가 있다. PyTorch는 실제 변화도가 아닌 야코비안 곱(jacobian product)를 계산한다.  
![pic](./img2.png)

In [13]:
inp = torch.eye(5, requires_grad=True)
out = (inp + 1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print(f'First call\n{inp.grad}')

out.backward(torch.ones_like(inp), retain_graph=True)
print(f'\nSecond call\n{inp.grad}')

inp.grad.zero_()
out.backward(torch.ones_like(inp),retain_graph=True)

print(f'\nCall after zeroing gradients\n{inp.grad}')

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


동일한 인자로 backward를 두 차례 호출하면 변화도 값이 달라진다. 
  
    
    이는 역전파를 수행 시, PyTorch가 변화도를 accumulate하기 때문이다. 
즉 계산된 변화도의 값이 연산 그래프의 모든 잎 노드의 grad 속성에 추가 된다.  
  
  
    따라서, 제대로 된 변화도를 계산하기 위해서는 grad 속성을 먼저 0으로 만들어야 한다.
  실제 학습 과정에서는 optimizer가 이 과정을 도와준다.